Import some possible required packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as scp
from scipy.spatial import Voronoi, voronoi_plot_2d
from scipy.interpolate import interp2d, LinearNDInterpolator
import movies_from_plots as mfp
import selfpropelledparticlevoronoi as sppv
import pathlib
from scipy import ndimage as nd 
import h5py as h5
import importlib

In [ ]:
importlib.reload(mfp)
importlib.reload(sppv)

We can make a regular (crystalline) lattice by following a "Bravais lattice" - like approach, where the lattice is a set of translations of points in 2D,

$R =  \sum_{i}{n_i \vec{a}_i} $

Where $n_i$ is an integer and $a_i$ is a basis vector of the lattice space. Our approach is slightly different, as we use nonintegers, that can be represented as 

$$ R = A \vec{u} $$

Where the columns of $A$ correspond to our basis vectors, and $\vec{u}$ is a vector of coordinates such that 

$u_i = n_i \Delta x$ 

where $\Delta x$ is the interval between the points such that all grid elements are equally spaced.

A model for T1 transition might possibly be:

Consider a graph with regions $R = \{R_\mu\}$, vertices $V = \{v_i\}$, and edges $E = \{(v_i,v_j)| v_i, v_j \in V\}$ where a region $R_\mu = \{(v_i,c_\mu)|v_i \in V\}$ for a center $c_\mu$

for 2 vertices $A$ and $B$, if their distance $|\vec{AB}|$ is less than or equal to a threshold value, they change their coordinate positions to: 

$$
(A',B') = \frac{\sqrt{3}}{2}(R_{\pm \pi/2}\vec{AB}+1/2\vec{AB}) +\vec{A}
$$

The transition can be modelled in two different ways:

1 - We describe a simple discrete model where 

$$
P[(A,B)\rightarrow(A',B')]=
\begin{cases}
0, & \text{for } |\vec{AB}| > l_\text{thresh}\\
1,  & \text{for } |\vec{AB}| \leq l_\text{thresh}
\end{cases}
$$

2 - Or we can follow a more continuous, physically inspired approach, where:

$$
P[(A,B)\rightarrow(A',B')] \propto \exp{(-\beta k |\vec{AB}|²)}
$$

3 - A mixed approach could be helpful to ensure the transition occurs at a reasonable rate.

$$
P[(A,B)\rightarrow(A',B')]=
\begin{cases}
0, & \text{for } |\vec{AB}| > l_\text{thresh}\\
\exp{(-\beta k |\vec{AB}|²)},  & \text{for } |\vec{AB}| \leq l_\text{thresh}
\end{cases}
$$

Where $\beta$ is meant to represent thermal energy, or some energy coefficient threshold. 

If we impose the criteria that 

$P[(A,B)\rightarrow(A',B')| \leq l_\text{thresh}] \geq  0.5$ 

we have the following inequality:

$$\beta \leq  \frac{\ln(2)}{k l_\text{thresh}²} = \beta_0$$


Possibly it is also necessary to change regions by considering the following statement.

A vertex $v_i$ has a neighbourhood of vertices $N_i$. The set of regions adjacent to $v_i$ is given by:

$$
S_i = \bigcup_{j,k \in N_i}{S_j\cap S_k}
$$

The model implemented is of the form:

For a center $c_i$

$$F_i = \sum_{v_\mu \in N_i}{f_{\mu i}}$$

For a vertex $v_\mu$
$$F_\mu = \sum_{c_i \in N_\mu}{f_{i\mu}} + \sum_{v_\nu \in N_\mu}{f_{\nu\mu}}$$ 

Where 

$$f_{ab} = -k(r_{ab}-r_0)\hat{n}_{ab}$$

In [1]:
import runspringnetsimul as rsns

45
0.096829875270601
done


In [1]:
import makefiguresandgraphics as mfag
#mfag
17%15

2

In [2]:
import makenetworkmovie as mnwm
#importlib.reload(mnwm)

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [ ]:
import movieforcefieldmovie as mffm
mffm

In [ ]:
filename = input('Video file name (with .mp4 included): ')
img_array,size = mfp.loadframe(0,500,1,'f')
mfp.savevideo(img_array,filename,size,'f')

# Insert test code here

## Make network